<a href="https://colab.research.google.com/github/crystalloide/RAG/blob/main/rag_langchain_2026.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# RAG with LangChain

| Step | Tech | Execution |
| --- | --- | --- |
| Embedding | Hugging Face / Sentence Transformers | 💻 Local |
| Vector store | Milvus | 💻 Local |
| Gen AI | Hugging Face Inference API | 🌐 Remote |

## Vue d'ensemble
Ce notebook implémente un système RAG qui permet de poser des questions sur des documents PDF en utilisant :

- Docling pour extraire et découper les documents
- Milvus comme base de données vectorielle
- Hugging Face pour les embeddings et le modèle de génération

Cet exemple exploite l'intégration
[LangChain Docling](../../integrations/langchain/), ainsi qu'une base de données vectorielles Milvus,
et des sentence-transformers embeddings.

Le composant `DoclingLoader` permet :

- d'utiliser facilement et rapidement différents types de documents dans les applications LLM, et
- d'exploiter le format riche de Docling pour un ancrage avancé et natif du document.

`DoclingLoader` prend en charge deux modes d'exportation :

- `ExportType.MARKDOWN` : si on souhaite collecter chaque document d'entrée comme un document LangChain distinct,

ou
- `ExportType.DOC_CHUNKS` (par défaut) : si on souhaite découper chaque document d'entrée en shards (segments) et

capturer ensuite chaque shard comme un document LangChain distinct.

L'exemple permet d'explorer les deux modes via le paramètre `EXPORT_TYPE` ; selon la valeur définie, le pipeline de l'exemple est configuré en conséquence.

## Configuration :

## Le notebook configure l'environnement avec :
- Les dépendances nécessaires (LangChain, Docling, Milvus, etc.)
- Les paramètres clés :
  - FILE_PATH : Le document PDF à analyser (ici, le rapport technique de Docling)
  - EMBED_MODEL_ID : Le modèle pour créer les embeddings des textes
  - GEN_MODEL_ID : Le modèle LLM pour générer les réponses (Mistral-7B)
  - EXPORT_TYPE : Mode de découpage des documents (DOC_CHUNKS ou MARKDOWN)
  - QUESTION : La question à poser au système

👉 Pour une vitesse de conversion optimale, utilisez l'accélération GPU quand c'est possible ; par exemple, si vous utilisez Colab, utilisez un environnement d'exécution compatible GPU.

Ce notebook utilise l'API d'inférence de Hugging Face ; pour augmenter le quota LLM, on peut fournir un jeton via la variable d'environnement HF_TOKEN.

Les dépendances peuvent être installées comme indiqué ci-dessous (l'option `--no-warn-conflicts` est destinée à l'environnement Python préconfiguré de Colab ; on peut la supprimer pour une utilisation plus stricte).

In [ ]:
%pip install -q --progress-bar off --no-warn-conflicts  langchain-classic langchain-docling langchain-core langchain-huggingface langchain_milvus pymilvus[milvus_lite] langchain python-dotenv

In [ ]:
import os
from pathlib import Path
from tempfile import mkdtemp

from dotenv import load_dotenv
from langchain_core.prompts import PromptTemplate
from langchain_docling.loader import ExportType


def _get_env_from_colab_or_os(key):
    try:
        from google.colab import userdata

        try:
            return userdata.get(key)
        except userdata.SecretNotFoundError:
            pass
    except ImportError:
        pass
    return os.getenv(key)


load_dotenv()

# https://github.com/huggingface/transformers/issues/5486:
os.environ["TOKENIZERS_PARALLELISM"] = "false"

HF_TOKEN = _get_env_from_colab_or_os("HF_TOKEN")
FILE_PATH = ["https://arxiv.org/pdf/2408.09869"]  # Docling Technical Report
EMBED_MODEL_ID = "sentence-transformers/all-MiniLM-L6-v2"
#GEN_MODEL_ID = "mistralai/Mixtral-8x7B-Instruct-v0.1"
GEN_MODEL_ID = "mistralai/Mistral-7B-Instruct-v0.2"
EXPORT_TYPE = ExportType.DOC_CHUNKS
QUESTION = "Which are the main AI models in Docling?"
PROMPT = PromptTemplate.from_template(
    "Context information is below.\n---------------------\n{context}\n---------------------\nGiven the context information and not prior knowledge, answer the query.\nQuery: {input}\nAnswer:\n",
)
TOP_K = 3
MILVUS_URI = str(Path(mkdtemp()) / "docling.db")

## Chargement des documents

Nous pouvons maintenant instancier notre chargeur et charger les documents.

DoclingLoader télécharge et parse le PDF

Le document est automatiquement découpé en "chunks" (morceaux) selon la stratégie choisie
- Deux modes possibles :
  - DOC_CHUNKS : Découpage intelligent en segments
  - MARKDOWN : Conversion en Markdown puis découpage par en-têtes

In [ ]:
from langchain_docling import DoclingLoader

from docling.chunking import HybridChunker

loader = DoclingLoader(
    file_path=FILE_PATH,
    export_type=EXPORT_TYPE,
    chunker=HybridChunker(tokenizer=EMBED_MODEL_ID),
)

docs = loader.load()

> Remarque : le message indiquant  `"Token indices sequence length is longer than the specified
maximum sequence length..."`  c'est-à-dire  `"La longueur de la séquence des indices de jetons est supérieure à la
longueur de séquence maximale spécifiée… "`  peut être ignoré dans ce cas — détails

[ici](https://github.com/docling-project/docling-core/issues/119#issuecomment-2577418826).

Détermination des splits (découpage ou splitting) :

Selon le mode choisi :
- Mode DOC_CHUNKS : Les chunks sont les docs (déjà prêts)
- Mode MARKDOWN : Utilise MarkdownHeaderTextSplitter pour découper selon les titres (#, ##, ###)

Cela crée une liste de splits - des petits morceaux de texte qui seront indexés.

In [ ]:
if EXPORT_TYPE == ExportType.DOC_CHUNKS:
    splits = docs
elif EXPORT_TYPE == ExportType.MARKDOWN:
    from langchain_text_splitters import MarkdownHeaderTextSplitter

    splitter = MarkdownHeaderTextSplitter(
        headers_to_split_on=[
            ("#", "Header_1"),
            ("##", "Header_2"),
            ("###", "Header_3"),
        ],
    )
    splits = [split for doc in docs for split in splitter.split_text(doc.page_content)]
else:
    raise ValueError(f"Unexpected export type: {EXPORT_TYPE}")

Regardons quelques splits exemple :

In [ ]:
for d in splits[:3]:
    print(f"- {d.page_content=}")
print("...")

## Ingestion

## Cette étape est cruciale :

- Chaque split est transformé en vecteur numérique (embedding) via le modèle sentence-transformers
- Ces vecteurs sont stockés dans Milvus (base de données vectorielle locale)

Cela permet de faire des recherches par similarité sémantique

In [ ]:
import json
from pathlib import Path
from tempfile import mkdtemp

from langchain_huggingface.embeddings import HuggingFaceEmbeddings
from langchain_milvus import Milvus

embedding = HuggingFaceEmbeddings(model_name=EMBED_MODEL_ID)


milvus_uri = str(Path(mkdtemp()) / "docling.db")  # or set as needed
vectorstore = Milvus.from_documents(
    documents=splits,
    embedding=embedding,
    collection_name="docling_demo",
    connection_args={"uri": milvus_uri},
    index_params={"index_type": "FLAT"},
    drop_old=True,
)

## RAG

## La chaîne RAG fonctionne ainsi :

- Retrieval (Récupération) :
  - La question est convertie en vecteur
  - Les TOP_K (3) chunks les plus similaires sont récupérés de Milvus


- Augmentation :
  - Ces chunks sont injectés dans un prompt avec la question
  - Le prompt structure : "Voici le contexte... Réponds à la question..."


- Generation :
  - Le LLM (Mistral-7B via l'API Hugging Face) génère une réponse basée sur le contexte fourni

In [ ]:
from langchain_classic.chains import create_retrieval_chain
from langchain_classic.chains.combine_documents import create_stuff_documents_chain
from langchain_huggingface import HuggingFaceEndpoint, ChatHuggingFace

retriever = vectorstore.as_retriever(search_kwargs={"k": TOP_K})

# Créer l'endpoint avec task conversational
endpoint = HuggingFaceEndpoint(
    repo_id=GEN_MODEL_ID,
    huggingfacehub_api_token=HF_TOKEN,
    task="conversational",
    max_new_tokens=512,
    temperature=0.7,
    top_p=0.95,
)

# Wrapper pour compatibilité avec les chaînes LangChain
llm = ChatHuggingFace(llm=endpoint)


def clip_text(text, threshold=100):
    return f"{text[:threshold]}..." if len(text) > threshold else text

## En retour, le système :
- Pose la question "Which are the main AI models in Docling?"
- Récupère les passages pertinents du PDF
- Génère une réponse basée sur ces passages
- Affiche la réponse + les sources utilisées avec leurs métadonnées

In [ ]:
question_answer_chain = create_stuff_documents_chain(llm, PROMPT)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)
resp_dict = rag_chain.invoke({"input": QUESTION})

clipped_answer = clip_text(resp_dict["answer"], threshold=200)
print(f"Question:\n{resp_dict['input']}\n\nAnswer:\n{clipped_answer}")
for i, doc in enumerate(resp_dict["context"]):
    print()
    print(f"Source {i + 1}:")
    print(f"  text: {json.dumps(clip_text(doc.page_content, threshold=350))}")
    for key in doc.metadata:
        if key != "pk":
            val = doc.metadata.get(key)
            clipped_val = clip_text(val) if isinstance(val, str) else val
            print(f"  {key}: {clipped_val}")

## Avantages de cette approche :

✅ Répond avec des informations spécifiques au document

✅ Cite les sources utilisées

✅ Évite les hallucinations en se basant sur du contenu réel

✅ Peut traiter des documents longs sans dépasser les limites de contexte du LLM
